In [8]:
from pathlib import Path
import numpy as np
from sklearn.externals import joblib
from keras.preprocessing import image
from keras.applications import xception

# Empty lists to hole the images and labels for each image
x_train = []
y_train = []

# Load the training data set by looping over every image file
for image_file in Path("training_dataset").glob("**/*.png"):
    
    # Loaf the current image file 
    image_data = image.load_img(
        image_file,
        target_size=(73, 73)
    )
    
    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)
    
    # Add the current image to our list of training images 
    x_train.append(image_array)
    
    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)
        
# Convert the list of seperate images into a single 4D numpy array. This is what Keras expects
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(
    weights="imagenet",
    include_top=False,
    input_shape=(73, 73, 3)
)
        
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file 
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

***Training our new classifier***

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from sklearn.externals import joblib 

# Load data set of extracted features 
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

In [12]:
# Create a model and add layers
model1= Sequential()

# Add layers to our model
model1.add(Flatten(input_shape=x_train.shape[1:]))
model1.add(Dense(128, activation="relu"))
model1.add(Dropout(0.5))
model1.add(Dense(1, activation = "sigmoid"))

# Compile the model
model1.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
# Train the model
model1.fit(
    x_train,
    y_train,
    validation_split=0.05,
    epochs=10,
    shuffle=True,
    verbose=2
)

# Save the trained model to a file so we can use it to make predictions later
model1.save("bird_feature_classifier_model.h5")

Instructions for updating:
Use tf.cast instead.
Train on 1900 samples, validate on 100 samples
Epoch 1/10
 - 7s - loss: 0.7410 - acc: 0.7526 - val_loss: 0.4296 - val_acc: 0.6900
Epoch 2/10
 - 2s - loss: 0.3852 - acc: 0.8211 - val_loss: 0.2598 - val_acc: 0.8700
Epoch 3/10
 - 2s - loss: 0.3475 - acc: 0.8495 - val_loss: 0.2887 - val_acc: 0.8400
Epoch 4/10
 - 2s - loss: 0.2910 - acc: 0.8642 - val_loss: 0.4343 - val_acc: 0.7700
Epoch 5/10
 - 2s - loss: 0.2873 - acc: 0.8679 - val_loss: 0.3508 - val_acc: 0.8100
Epoch 6/10
 - 2s - loss: 0.2506 - acc: 0.8900 - val_loss: 0.4933 - val_acc: 0.7700
Epoch 7/10
 - 2s - loss: 0.2266 - acc: 0.9005 - val_loss: 0.3882 - val_acc: 0.8000
Epoch 8/10
 - 2s - loss: 0.1928 - acc: 0.9200 - val_loss: 0.4104 - val_acc: 0.8300
Epoch 9/10
 - 2s - loss: 0.1869 - acc: 0.9200 - val_loss: 0.5151 - val_acc: 0.7900
Epoch 10/10
 - 2s - loss: 0.1601 - acc: 0.9300 - val_loss: 0.6262 - val_acc: 0.7400


***Using the Model to make Predictions***

In [14]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from keras.applications import xception 

In [15]:
image_to_test = "bird_1.png"

# Load the model we trained 
model2 = load_model("bird_feature_classifier_model.h5")

# Load image to test, resizing it to 73 pixels (as required by this model)
img = image.load_img(image_to_test, target_size=(73, 73))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
images = np.expand_dims(image_array, axis=0)


In [16]:
# Normalize the data 
images = xception.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = xception.Xception(
    weights="imagenet",
    include_top=False,
    input_shape=(73, 73, 3)
)

features = feature_extraction_model.predict(images)

In [18]:
# Given the extracted features, make a final prediction using our own model
results = model2.predict(features)

# Since we are only testing one image with possible class, we only need to check the first result's first element
single_result = results[0][0]

# Print the result
print(f"Likelihood that {image_to_test} is a bird: {single_result*100}%")

Likelihood that bird_1.png is a bird: 99.84486103057861%


***Checking Precision and Recall***

In [19]:
import numpy as np
from keras.preprocessing import image
from keras.applications import xception
from pathlib import Path
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report 

In [21]:
# Empty lists to hold the images and labels for rach image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("test_dataset").glob("**/*.png"):
    
    # Load the current image file
    image_data = image.load_img(
        image_file,
        target_size = (73, 73)
    )
    
    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)
    
    # Add the current image to our list of test images
    x_test.append(image_array)
    
    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)
        
# Convert the list of test images to a numpy array
x_test = np.array(x_test)
    
# Normalize test data st to 0-to-1 range
x_test = xception.preprocess_input(x_test)

In [22]:
# Load the Xception neural network to use as a feature extractor
feature_extractor = xception.Xception(
    weights="imagenet",
    include_top=False,
    input_shape=(73, 73, 3)
)

# Load our trained classifier model
model3 = load_model("bird_feature_classifier_model.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model
predictions = model3.predict(features_x)

# If the model is more than 50% sure the object is a bird, call it a bird. Otherwise, call it "not a bird"
predictions = predictions > 0.5 

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

True Positives: 91
True Negatives: 76
False Positives: 24
False Negatives: 9
             precision    recall  f1-score   support

          0       0.89      0.76      0.82       100
          1       0.79      0.91      0.85       100

avg / total       0.84      0.83      0.83       200

